In [2]:
import pandas as pd
import numpy as np

# 開始日の指定
start = '20210905'
end = '20211004'

# この団体には学生支部と社会人支部の2つのグループがある
path1 = f'学生支部-{start}-{end}.csv'
path2 = f'社会人支部-{start}-{end}.csv'
df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)
dfs = [df1, df2]
# df
# col = df.columns
# col

# 日付データの処理
def convert_date(df, col):
    df = df.copy()
    df[col] = pd.to_datetime(df[col])
    #df['日付'] = df[col].map(lambda x:str(x.year)+str(x.month).zfill(2)+str(x.day).zfill(2))
    df['日付'] = df[col].map(lambda x:str(x.month)+'/'+str(x.day))
    
    #df['月'] = df[col].map(lambda
    # df['日'] = df[col].map(lambda x:x.day)
    df = df.drop(col, axis=1)
    return df

def data_shaping(df):
    df_ = df.copy()
    
    # 不要なカラムを削除
    drops = ["'ID'", "'グループ名'", "'発熱'", "'平熱'", "'最終出勤日'", 
         "'体温(問診票情報)'", "'症状(問診票情報)'", 
         "'通勤手段（選択）'", "'通勤手段（詳細）'",
        "'受診医療機器名'", "'PCR検査（有無）'", 
         "'PCR検査（詳細）'", "'感染者と接触有無'", 
         "'感染者接触（詳細）'", "'県外移動（有無）都道府県外移動、海外渡航 履歴'",
         "'県外移動（詳細：誰が）'", "'県外移動（詳細：場所）'", 
        "'県外移動（期間）'", "'人の過密場所への移動(有無)'", 
         "'人の過密場所への移動（詳細：誰が）'",
        "'人の過密場所への移動（詳細：場所）'", 
         "'人の過密場所への移動(期間表示）'"]
    
    df_ = df_.drop(drops, axis=1)
    
    # カラムの名前を変更
    renames = {"'氏名'":"氏名", 
           "'日付・時刻'":"日付・時刻",
           "'体温(体調情報)'":"体温",
           "'症状(体調情報)'":"症状",
           "'メッセージ'":"メッセージ"}
    df_ = df_.rename(columns=renames)
    
    #日付・時刻→日付
    df_ = convert_date(df_, '日付・時刻')
    
    # 引用符をなくす
    cols = ['氏名', '体温', '症状', 'メッセージ']
    for c in cols:
        df_[c] = df_[c].str[1:]
        df_[c] = df_[c].str[:-1]
        
    return df_


df1 = data_shaping(df1)
df2 = data_shaping(df2)
df2 = df2[df2['氏名'] != '代表者氏名１']
df2 = df2[df2['氏名'] != '代表者氏名2']
df_ = pd.concat([df1, df2])
# df_

members = [
'苗字 一郎'
'苗字 二郎'
'苗字 三郎'
'苗字 四郎'
'苗字 五郎'
'苗字 六郎'
'苗字 七郎']


#名前×日付データ
def name_day(df_, members=members):
    # 名簿
#     members = []
#     for m in df_['氏名'].unique():
#         members.append(m)

    # 日付一覧
    days = []
    for d in df_['日付'].unique():
        days.append(d)

    # http://hxn.blog.jp/archives/11591404.html
    A = pd.DataFrame(index=members, columns=days)

    # 37.5度以上があったら、体温表示、無ければ〇
    for m in members:
        personal_df = df_[df_.氏名==m]
        
        for i in range(len(personal_df)):
            # 高熱の場合は★をつける
            if float(personal_df.iloc[i]['体温']) > 37.4:
                personal_df.iloc[i]['体温'] = '★' + personal_df.iloc[i]['体温']

        for d in A.columns:
            day_data = personal_df[personal_df.日付==d]

            if day_data.empty is True:
                pass

            else:
                text = day_data.iloc[0]['体温']
                A.loc[m][d] = text   
            
    return A

A = name_day(df_)

A.insert(0, '未記録数','0')
#A

a = A.isnull().sum(axis=1)
for i in range(len(a)):
    b = a.iloc[i]
    if b == 0:
        A.iat[i,0] = '〇'
    else:
        A.iat[i,0] = b
#A

# insertは、列には使えるが行には使えないので、
# 転置→insert→転置する必要がある

B = A.copy()
AT = A.T

# 学年の人数により、indexを手動で設定
# 個人情報保護のため、数字は適当に変えています

# 学生支部
AT.insert(0, '大学1年', '----')
AT.insert(10, '大学2年', '----')
AT.insert(25, '大学3年', '----')
AT.insert(40, '大学4年', '----')
AT.insert(60, '大学院生', '----')
# 社会人支部
AT.insert(80, '社会人', '----')
A = AT.T

# 学生支部と社会人支部を分ける
A1 = A.iloc[:-20]
A2 = A.iloc[-20:]
# A1

term = f'{start}_{end}'
path = f'団体名_体温記録_{term}.xlsx'

with pd.ExcelWriter(path) as writer:
    A1.to_excel(writer, sheet_name=f"学生支部")
    A2.to_excel(writer, sheet_name=f"社会人支部")

B1 = B.iloc[:10]
B2 = B.iloc[10:10+15]
B3 = B.iloc[10+15:10+15+15]
B4 = B.iloc[10+15+15:10+15+15+20]
B5 = B.iloc[10+15+15+20:10+15+15+20+20]
B6 = B.iloc[-20:]

# OK(2週間分の記録が全て提出されている)とMISSING(提出不足がある)を判断
def ok_missing(A):  
    OK = []
    MISSING = []

    for i in A.index:
        if A.loc[i].isna().sum() == 0:
            OK.append(i)
        else:
            MISSING.append(i)
    
    return OK, MISSING

O1, M1 = ok_missing(B1)
O2, M2 = ok_missing(B2)
O3, M3 = ok_missing(B3)
O4, M4 = ok_missing(B4)
O5, M5 = ok_missing(B5)
O6, M6 = ok_missing(B6)

def ok_display(O):
    j = 0
    k = []
    for i in O:
        if j == 5:
            print(k)
            j = 0
            k = []

        k.append(i)
        j += 1
    print(k)

print(f'{start}-{end}')
print()
print('1年:', len(O1))
ok_display(O1)
print()
print('2年:', len(O2))
ok_display(O2)
print()
print('3年:', len(O3))
ok_display(O3)
print()
print('4年:', len(O4))
ok_display(O4)
print()
print('大学院生:', len(O5))
ok_display(O5)
print()
print('社会人支部:', len(O6))
ok_display(O6)
print()

In [5]:
print('出てない人リスト')
print(f'{start}-s{end}')
print()
print('1年:', len(M1))
ok_display(M1)
print()
print('2年:', len(M2))
ok_display(M2)
print()
print('3年:', len(M3))
ok_display(M3)
print()
print('4年:', len(M4))
ok_display(M4)
print()
print('院生:', len(M5))
ok_display(M5)
print()
print('社会人支部:', len(M6))
ok_display(M6)
print()